<h1>3. Skill Prediction en LinkedIN

**3.0 Importaciones necesarias**

Se importan librerías y métodos necesarios para el correcto funcionamiento de las activades a desarrollar dentro de ésta sección.

In [1]:
#!usr/bin/env python
# -*- coding: utf-8 -*-
from scipy.sparse import csr_matrix
import re
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.utils import column_or_1d
from clasificadores import *

**3.1 Construcción de dataset**

Primeramente, se crea la función *generar_Z*, la cual, a partir de los archivos *user_skill* y *skill_id*, construye una matriz dispersa con los datos provistos en dicho archivo. Tal matriz será denotada de aquí en adelante como Z.

In [2]:
#Se define funcion para construir matriz Z
def generar_Z(arch1, arch2):
    #Variables que contienen la cantidad de perfiles y de competencias, respectivamente
    n_perfiles = 0
    n_competencias = 0
    
    #Se determina la cantidad de perfiles
    with open(arch1) as perfiles:
        n_perfiles = sum(1 for _ in perfiles)
    
    #Se determina la cantidad de competencias
    with open(arch2) as competencias:
        n_competencias = sum(1 for _ in competencias)
        
    #Se crea fisicamente la matriz Z
    Z = np.zeros(shape=(n_perfiles, n_competencias), dtype=np.int)
    
    #Se procede a 'trasladar' los valores de los archivos a Z
    with open(arch1) as perfiles:
        for line in perfiles:
            datos = re.split(':|,|\n', line)
            n_perfil = int(datos[0])
            for n_competencia in datos[1:-1]:
                Z[n_perfil, int(n_competencia)] = 1
    
    Z = csr_matrix(Z)
    return Z

Luego, se procede a construir la matriz Z con que se trabajará de ahora en adelante.

In [3]:
Z = generar_Z('user_skill', 'skill_id')

A partir del resultado anterior, se puede obtener la cantidad de perfiles y de competencias que integran los dataset.

In [4]:
print 'Se cuenta con un dataset compuesto por', Z.shape[0], 'perfiles y', Z.shape[1], 'competencias.'

Se cuenta con un dataset compuesto por 7890 perfiles y 14544 competencias.


**3.2 Construcción de set de entrenamiento y set de pruebas**

A partir de la matriz Z generada en la sección anterior, se procede a construir los conjuntos de entrenamiento y de prueba. Para ello, se seleccionan registros desde Z en forma azarosa de tal forma que el conjunto de entrenamiento estará compuesto por el 60% de los registros del dataset original, mientras que el conjunto de prueba estará integrando por el 40% de los registros de dicho dataset. Considerando que Z acumula 7890 perfiles, el conjunto de entrenamiento tendrá 4734 registros y el de prueba, 3156 registros.

Para realizar la tarea descrita, se sigue el procedimiento presentado a continuación:

In [5]:
#Se permutan aleatoriamente las filas de la matriz Z
Z = shuffle(Z, random_state=0)

#Se construyen conjunto de entrenamiento y de prueba, respectivamente
Z_tr = Z[0:int(0.6*Z.shape[0])]
Z_ts = Z[int(0.6*Z.shape[0]):]

**3.3 Cantidad de perfiles por competencia**

En ésta sección, se desea construir un gráfico en el que sea posible apreciar la cantidad de perfiles que declaran una determinada competencia. Para esto, se lleva a cabo el siguiente procedimiento. Notar que se han ignorado aquellas competencias cuyo nombre es desconocido, pues no están entregando información relevante acerca del problema a un lector externo.    

In [6]:
#NOTA: La siguiente pieza de codigo demora alrededor de 20 segundos en ejecutarse

#Se obtiene el nombre, identificador y cantidad de perfiles asociado a cada competencia, y se 
#almacenan en una lista de tuplas

lista_competencias = []
with open('skill_id') as competencias:
    for line in competencias:
        dato = re.split(' :|:|\n', line)
        dato = dato[0:-1]
        
        if (len(dato) > 2):
            nombre_competencia = ''
            for token in dato[0:-1]:
                nombre_competencia += token
            id_competencia = int(dato[-1])
            cantidad_perfiles = Z[:, id_competencia].sum()
            lista_competencias.append((nombre_competencia + '(' + str(id_competencia) + ')', cantidad_perfiles))
            
        else:
            if (dato[0] != ''):
                nombre_competencia = dato[0]
                id_competencia = int(dato[1])
                cantidad_perfiles = Z[:, id_competencia].sum()
                lista_competencias.append((nombre_competencia + '(' + str(id_competencia) + ')', cantidad_perfiles))
            
#Finalizado el proceso anterior, se ordena la lista competencias por su frecuencia
lista_competencias.sort(key=lambda tup: tup[1], reverse=True)

#Finalmente, se elabora gráfico que muestra las 10 competencias mas frecuentes, junto con su
#respectiva frecuencia
top_competencias = lista_competencias[0:10]
labels = zip(*top_competencias)[0]
labels = [unicode(label, "utf-8") for label in labels][::-1]
frecuencias = zip(*top_competencias)[1]
frecuencias = [frecuencia for frecuencia in frecuencias][::-1]
x_pos = np.arange(len(frecuencias))
plt.barh(x_pos, frecuencias,align='center', height=0.1)
plt.yticks(x_pos, labels)
plt.title('Cantidad de perfiles que declaran una competencia')
plt.xlabel(u'Número de perfiles')
plt.ylabel('Competencia(id)')
plt.tight_layout()
plt.show()

**3.4 Predicción de la competencia *Microsoft Office***

**3.4.1 Ajuste de conjuntos de entrenamiento y prueba**

Se desea predecir la existencia de la competencia *Microsoft Office* en un perfil determinado, en base a las competencias restantes que el usuario declaró. Se ha seleccionado dicha competencia debido a su alta frecuencia de aparición en los perfiles. Para el posterior entrenamiento de clasificadores, se ajustan los sets de entrenamiento y de prueba construídos previamente para así poder abordar este problema. Notar que para evaluar el rendimiento de cada clasificador, se usarán las funciones previamente creadas para abordar el problema 2 junto la función *KNN*, que permite entrenar un modelo Vecinos Más Cercanos. Además, se usa la siguiente notación:

clase 1: Implica que la competencia *Microsoft Office* efectivamente es declarada en el perfil actual.  
clase 0: Caso contrario.

Se procede de la siguiente manera:

In [44]:
#Para el set de entrenamiento, se construyen las matrices X_tr e y_tr a partir de Z_tr
#Recordar que Microsoft Office corresponde a la competencia numero 29
y_tr = column_or_1d(Z_tr[:, 29].toarray())
X_tr = csr_matrix(np.delete(Z_tr.toarray(), 29, 1))

#Se procede de forma analoga con el conjunto de entrenamiento, construyendo X_ts e y_ts
#a partir de Z_ts
y_ts = column_or_1d(Z_ts[:, 29].toarray())
X_ts = csr_matrix(np.delete(Z_ts.toarray(), 29, 1))

**3.4.2 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Binario**

En base a los conjuntos de datos elaborados en la sección anterior, se procede a entrenar/ajustar un clasificador Bayesiano Binario. Posterior a ello, se aplica el modelo generado sobre los datos de entrenamiento y de prueba con el fin de determinar su precisión sobre estos conjuntos.

In [43]:
NAIVE_BAYES(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento BernoulliNB: 0.849599
Precisión datos de prueba BernoulliNB: 0.794360
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.81      0.87      0.84      1914
    clase 0       0.77      0.68      0.72      1242

avg / total       0.79      0.79      0.79      3156



BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

**3.4.3 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Multinomial**

Se ejecuta el mismo procedimiento de la sección previa, pero considerando un clasificador Bayesiano Ingenuo Multinomial.

In [9]:
MULTINOMIAL(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento MULTINOMIAL: 0.847909
Precisión datos de prueba MULTINOMIAL: 0.786122
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.83      0.81      0.82      1914
    clase 0       0.72      0.75      0.73      1242

avg / total       0.79      0.79      0.79      3156



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**3.4.4 Entrenamiento/ajuste de modelo de Regresión Logística Regularizado** 

Se procede de manera análoga a las dos secciones anteriores, entrenando un clasificador de regresión logística bajo el efecto de variados parámeteros de regularización C.

In [ ]:
LOGIT(X_tr,y_tr,X_ts,y_ts, 0)

Usando C= 0.010000
Precisión datos de entrenamiento LOGISTIC: 0.843684
Precisión datos de prueba LOGISTIC: 0.829848
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.83      0.91      0.87      1914
    clase 0       0.84      0.70      0.76      1242

avg / total       0.83      0.83      0.83      3156

Usando C= 0.100000
Precisión datos de entrenamiento LOGISTIC: 0.880228
Precisión datos de prueba LOGISTIC: 0.826679
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.83      0.90      0.86      1914
    clase 0       0.82      0.72      0.77      1242

avg / total       0.83      0.83      0.82      3156

Usando C= 10.000000
Precisión datos de entrenamiento LOGISTIC: 0.984157
Precisión datos de prueba LOGISTIC: 0.791191
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    c

**3.4.5 Entrenamiento/ajuste de una SVM Lineal**

Se entrena una SVM lineal bajo el efecto de variados parámetros de regularización C.

In [11]:
SVM(X_tr,y_tr,X_ts,y_ts, 0)

El valor de C que se esta probando: 0.010000
Precisión datos de entrenamiento SVM: 0.838192
Precisión datos de prueba SVM: 0.823194
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.81      0.93      0.86      1914
    clase 0       0.86      0.66      0.75      1242

avg / total       0.83      0.82      0.82      3156

El valor de C que se esta probando: 0.100000
Precisión datos de entrenamiento SVM: 0.911914
Precisión datos de prueba SVM: 0.814005
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.82      0.89      0.85      1914
    clase 0       0.80      0.70      0.75      1242

avg / total       0.81      0.81      0.81      3156

El valor de C que se esta probando: 10.000000
Precisión datos de entrenamiento SVM: 0.989227
Precisión datos de prueba SVM: 0.768378
Análisis detallado de resultados sobre set de prueba:
           

**3.4.6 Entrenamiento/ajuste de modelo *k-NN* **

Finalmente, se entrena un modelo k-NN para distintos valores de k. Notar que no es posible verificar el comportamiento de los modelos resultantes desde k = 1 hasta k = *cantidad_de_muestras* (en este caso, *cantidad_de_muestras* = 4734), dado a que sería un procedimiento muy costoso, debido a la gran cantidad de valores que k podría tomar. En consecuencia, se toman cinco valores para k, partiendo desde k = 1 y aumentando el valor de dicho parámetro a intervalos regulares. 

In [9]:
KNN(X_tr, y_tr, X_ts, y_ts, Z_tr.shape[0])

Usando k = 1.000000
Precisión datos de entrenamiento k-NN: 1.000000
Precisión datos de prueba k-NN: 0.726869
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.76      0.81      0.78      1914
    clase 0       0.67      0.60      0.63      1242

avg / total       0.72      0.73      0.72      3156

Usando k = 947.000000
Precisión datos de entrenamiento k-NN: 0.614913
Precisión datos de prueba k-NN: 0.609316
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.61      1.00      0.76      1914
    clase 0       1.00      0.01      0.01      1242

avg / total       0.76      0.61      0.46      3156

Usando k = 1893.000000
Precisión datos de entrenamiento k-NN: 0.611534
Precisión datos de prueba k-NN: 0.606464
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.61

**3.4.7 Análisis de resultados**

Para el modelo de regresión logística, se observa que la mejor precisión para el conjunto de prueba se consigue bajo el parámetro de regularización C = 0.01, aunque la precisión sobre el conjunto de entrenamiento alcanza su valor mínimo. Para las siguientes instancias de C, la precisión en entrenamiento comienza a aumentar, pero disminuye para las pruebas. Por otro lado, para el modelo de SVM lineal, se observa el mismo comportamiento descrito anteriormente, siendo también C = 0.01 cuando la precisión sobre el conjunto de pruebas alcanza su valor máximo. Se puede ver que, si bien los resultados de los modelos binario y multinomial son buenos, no logran superar ni a SVM ni a regresión logística.

Para k-NN, tanto la precisión de entrenamiento como la de prueba alcanzan sus valores más altos para k = 1. Más aún, tales valores de precisión disminuyen abruptamente para las siguientes instancias de k, obteniendose rendimientos bastante pobres para estos modelos.

Así, se ve que regresión logística, con C = 0.01, es en definitiva el método que predice en forma más precisa la presencia de la competencia *Microsoft Office* en un perfil dado, con precisión de entrenamiento 0.843684 y precisión de prueba 0.829848. Además, k-NN es quién consigue los peores resultados. 

**3.5 Predicción de la competencia *Liderazgo de equipos***

**3.5.1 Ajuste de conjuntos de entrenamiento y prueba**

Se desea predecir la existencia de la competencia *Liderazgo de equipos* (id = 32) en un perfil dado, en base a las competencias restantes que el usuario declaró. Notar que corresponde a la segunda competencia más frecuente. Los métodos de clasificación a evaluar corresponden a los mismos de la sección 3.4. Además, se usa la siguiente notación:

clase 1: Implica que la competencia *Liderazgo de equipos* efectivamente es declarada en el perfil actual.  
clase 0: Caso contrario.

Se procede de la siguiente manera:

In [49]:
#Para el set de entrenamiento, se construyen las matrices X_tr e y_tr a partir de Z_tr
#Recordar que Liderazgo de equipos corresponde a la competencia numero 32
y_tr = column_or_1d(Z_tr[:, 32].toarray())
X_tr = csr_matrix(np.delete(Z_tr.toarray(), 32, 1))

#Se procede de forma analoga con el conjunto de entrenamiento, construyendo X_ts e y_ts
#a partir de Z_ts
y_ts = column_or_1d(Z_ts[:, 32].toarray())
X_ts = csr_matrix(np.delete(Z_ts.toarray(), 32, 1))

**3.5.2 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Binario**

Se entrena y posteriormente se evalúa modelo Bayesiano Ingenuo Binario en base a los nuevos datos.

In [50]:
NAIVE_BAYES(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento BernoulliNB: 0.877905
Precisión datos de prueba BernoulliNB: 0.860900
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.89      0.90      0.89      2069
    clase 0       0.80      0.79      0.80      1087

avg / total       0.86      0.86      0.86      3156



BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

**3.5.3 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Multinomial**

Se entrena y evalúa modelo Bayesiano Ingenuo Multinomial en base a nuevos datos.

In [51]:
MULTINOMIAL(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento MULTINOMIAL: 0.861639
Precisión datos de prueba MULTINOMIAL: 0.818441
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.91      0.80      0.85      2069
    clase 0       0.69      0.85      0.76      1087

avg / total       0.84      0.82      0.82      3156



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**3.5.4 Entrenamiento/ajuste de modelo de Regresión Logística Regularizado** 

Se entrena y evalúa modelo de Regresión Logística Regularizado, bajo diferentes parámetros de regularización C, en base a nuevos datos.

In [52]:
LOGIT(X_tr,y_tr,X_ts,y_ts, 0)

Usando C= 0.010000
Precisión datos de entrenamiento LOGISTIC: 0.888466
Precisión datos de prueba LOGISTIC: 0.892269
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.91      0.93      0.92      2069
    clase 0       0.86      0.82      0.84      1087

avg / total       0.89      0.89      0.89      3156

Usando C= 0.100000
Precisión datos de entrenamiento LOGISTIC: 0.915927
Precisión datos de prueba LOGISTIC: 0.895120
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.91      0.93      0.92      2069
    clase 0       0.86      0.83      0.84      1087

avg / total       0.89      0.90      0.89      3156

Usando C= 10.000000
Precisión datos de entrenamiento LOGISTIC: 0.983523
Precisión datos de prueba LOGISTIC: 0.861217
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    c

**3.5.5 Entrenamiento/ajuste de una SVM Lineal**

Se entrena y se evalúa una SVM Lineal bajo diferentes parámetros de regularización C, en base a nuevos datos.

In [59]:
SVM(X_tr,y_tr,X_ts,y_ts, 0)

El valor de C que se esta probando: 0.010000
Precisión datos de entrenamiento SVM: 0.895437
Precisión datos de prueba SVM: 0.893219
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.91      0.93      0.92      2069
    clase 0       0.87      0.82      0.84      1087

avg / total       0.89      0.89      0.89      3156

El valor de C que se esta probando: 0.100000
Precisión datos de entrenamiento SVM: 0.941487
Precisión datos de prueba SVM: 0.891318
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.91      0.92      0.92      2069
    clase 0       0.85      0.83      0.84      1087

avg / total       0.89      0.89      0.89      3156

El valor de C que se esta probando: 10.000000
Precisión datos de entrenamiento SVM: 0.988804
Precisión datos de prueba SVM: 0.833016
Análisis detallado de resultados sobre set de prueba:
           

KeyboardInterrupt: 

**3.5.6 Entrenamiento/ajuste de modelo *k-NN* **

Se entrena y evalúa modelo k-NN siguiendo la misma lógica de la sección 3.4.6.

In [56]:
KNN(X_tr, y_tr, X_ts, y_ts, Z_tr.shape[0])

Usando k = 1.000000
Precisión datos de entrenamiento k-NN: 0.999155
Precisión datos de prueba k-NN: 0.776933
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.84      0.82      0.83      2069
    clase 0       0.67      0.70      0.68      1087

avg / total       0.78      0.78      0.78      3156

Usando k = 947.000000
Precisión datos de entrenamiento k-NN: 0.648711
Precisión datos de prueba k-NN: 0.657161
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.66      1.00      0.79      2069
    clase 0       1.00      0.00      0.01      1087

avg / total       0.77      0.66      0.52      3156

Usando k = 1893.000000
Precisión datos de entrenamiento k-NN: 0.646388
Precisión datos de prueba k-NN: 0.655577
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.66

**3.5.7 Análisis de resultados**

En primer lugar, se observa que en comparación a la predicción de la competencia *Microsoft Office*, se obtienen valores superiores de precisión, tanto de entrenamiento como de prueba, para todos los métodos de clasificación utilizados en la predicción de la competencia *Liderazgo de equipos*. 

Si se analiza cada clasificador en detalle, se tiene que éstos se comportan de similar forma respecto a la predicción de *Microsoft Office*: Regresión logística logra su mejor rendimiento para C = 0.1, al igual que la SVM lineal. Además, k-NN resulta ser el peor de los métodos, aunque mejora en comparación a la aplicación de k-NN en la predicción de *Microsoft Office*. Otra vez, tanto el modelo binario como el multinomial no logran mejorar los resultados de SVM y regresión logística. 

Comparando los cinco métodos, se tiene que nuevamente es regresión logística quien alcanza los mayores valores de precisión, pero ésta vez para C = 0.1, con precisión de entrenamiento 0.915927 y precisión de prueba 0.895120.

**3.6 Predicción de la competencia *Gestión de proyectos***

**3.6.1 Ajuste de conjuntos de entrenamiento y prueba**

Se desea predecir la existencia de la competencia *Gestión de proyectos* (id = 43) en un perfil dado, en base a las competencias restantes que el usuario declaró. Notar que corresponde a la tercera competencia más frecuente. Los métodos de clasificación a evaluar corresponden a los mismos de las secciones 3.4 y 3.5. Además, se usa la siguiente notación:

clase 1: Implica que la competencia *Gestión de proyectos* efectivamente es declarada en el perfil actual.  
clase 0: Caso contrario.

Se procede de la siguiente manera:

In [7]:
#Para el set de entrenamiento, se construyen las matrices X_tr e y_tr a partir de Z_tr
#Recordar que Gestión de proyectos corresponde a la competencia numero 43
y_tr = column_or_1d(Z_tr[:, 43].toarray())
X_tr = csr_matrix(np.delete(Z_tr.toarray(), 43, 1))

#Se procede de forma analoga con el conjunto de entrenamiento, construyendo X_ts e y_ts
#a partir de Z_ts
y_ts = column_or_1d(Z_ts[:, 43].toarray())
X_ts = csr_matrix(np.delete(Z_ts.toarray(), 43, 1))

**3.6.2 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Binario**

Se entrena y evalúa clasificador Bayesiano Ingenuo Binario con los nuevos datos generados.

In [9]:
NAIVE_BAYES(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento BernoulliNB: 0.861850
Precisión datos de prueba BernoulliNB: 0.834284
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.88      0.89      0.89      2278
    clase 0       0.70      0.70      0.70       878

avg / total       0.83      0.83      0.83      3156



BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

**3.6.3 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Multinomial**

Se entrena y evalúa clasificador Bayesiano Ingenuo Multinomial a partir de los nuevos datos generados.

In [11]:
MULTINOMIAL(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento MULTINOMIAL: 0.867765
Precisión datos de prueba MULTINOMIAL: 0.809886
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.92      0.80      0.86      2278
    clase 0       0.62      0.82      0.71       878

avg / total       0.84      0.81      0.82      3156



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

**3.6.4 Entrenamiento/ajuste de modelo de Regresión Logística Regularizado**

Se entrena y evalúa modelo de Regresión Logística Regularizado para diferentes parámetros de regularización C.

In [13]:
LOGIT(X_tr, y_tr, X_ts, y_ts, 0)

Usando C= 0.010000
Precisión datos de entrenamiento LOGISTIC: 0.875581
Precisión datos de prueba LOGISTIC: 0.873574
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.89      0.93      0.91      2278
    clase 0       0.81      0.72      0.76       878

avg / total       0.87      0.87      0.87      3156

Usando C= 0.100000
Precisión datos de entrenamiento LOGISTIC: 0.912759
Precisión datos de prueba LOGISTIC: 0.873891
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.90      0.93      0.91      2278
    clase 0       0.80      0.72      0.76       878

avg / total       0.87      0.87      0.87      3156

Usando C= 10.000000
Precisión datos de entrenamiento LOGISTIC: 0.982678
Precisión datos de prueba LOGISTIC: 0.838720
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    c

 **3.6.5 Entrenamiento/ajuste de una SVM lineal**

Se entrena y evalúa una SVM lineal para diferentes parámetros de regularización C.

In [14]:
SVM(X_tr, y_tr, X_ts, y_ts, 0)

El valor de C que se esta probando: 0.010000
Precisión datos de entrenamiento SVM: 0.888466
Precisión datos de prueba SVM: 0.871039
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.89      0.93      0.91      2278
    clase 0       0.81      0.71      0.75       878

avg / total       0.87      0.87      0.87      3156

El valor de C que se esta probando: 0.100000
Precisión datos de entrenamiento SVM: 0.937896
Precisión datos de prueba SVM: 0.871990
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.90      0.92      0.91      2278
    clase 0       0.79      0.74      0.76       878

avg / total       0.87      0.87      0.87      3156

El valor de C que se esta probando: 10.000000
Precisión datos de entrenamiento SVM: 0.987748
Precisión datos de prueba SVM: 0.808619
Análisis detallado de resultados sobre set de prueba:
           

**3.6.6 Entrenamiento/ajuste de modelo *k-NN***

Se entrena y evalúa un modelo k-NN para diferentes valores del parámetro k, al igual que como se hizo en las secciones 3.4.6 y 3.5.6.

In [15]:
KNN(X_tr, y_tr, X_ts, y_ts, Z_tr.shape[0])

Usando k = 1.000000
Precisión datos de entrenamiento k-NN: 0.998944
Precisión datos de prueba k-NN: 0.809886
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.84      0.91      0.87      2278
    clase 0       0.71      0.54      0.61       878

avg / total       0.80      0.81      0.80      3156

Usando k = 947.000000
Precisión datos de entrenamiento k-NN: 0.720110
Precisión datos de prueba k-NN: 0.722117
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.72      1.00      0.84      2278
    clase 0       1.00      0.00      0.00       878

avg / total       0.80      0.72      0.61      3156

Usando k = 1893.000000
Precisión datos de entrenamiento k-NN: 0.719899
Precisión datos de prueba k-NN: 0.721800
Análisis detallado de resultados sobre set de prueba:


/usr/lib64/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

    clase 1       0.72      1.00      0.84      2278
    clase 0       0.00      0.00      0.00       878

avg / total       0.52      0.72      0.61      3156

Usando k = 2839.000000
Precisión datos de entrenamiento k-NN: 0.719899
Precisión datos de prueba k-NN: 0.721800
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.72      1.00      0.84      2278
    clase 0       0.00      0.00      0.00       878

avg / total       0.52      0.72      0.61      3156

Usando k = 3785.000000
Precisión datos de entrenamiento k-NN: 0.719899
Precisión datos de prueba k-NN: 0.721800
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.72      1.00      0.84      2278
    clase 0       0.00      0.00      0.00       878

avg / total       0.52      0.72      0.61      3156

Usando k = 4731.000000


**3.6.7 Análisis de resultados**

La predicción de la competencia *Gestión de proyectos* alcanza, en general, precisiones inferiores a las alcanzadas en la predicción de *Liderazgo de equipos*, pero superiores a las de *Microsoft Office*. 

No hay novedades en lo que respecta al comportamiento de los clasificadores: Regresión logística alcanza su mejor rendimiento para C = 0.1, al igual que el modelo SVM. Una vez más, k-NN obtiene los peores resultados, pero mejora respecto a la predicción de las competencias previamente analizadas. Los modelos binario y multinomial nuevamente consiguen buenos resultados, pero no consiguen ser superiores ni a regresión logística ni a SVM.

En consecuencia, y por tercera vez consecutiva, es regresión logística quién obtiene los mejores resultados en cuanto a precisión, para C = 0.1, con precisión de entrenamiento 0.912759 y precisión de prueba 0.873574.